In [176]:
import mph   #导入mph模块，该模块为COMSOL接口提供支持   https://mph.readthedocs.io/en/stable/installation.html

path = 'E:\GIF+图片\优化算法\优化模型'
client = mph.start()   #启动COMSOL客户端
pymodel = client.load(path)    #导入COMSOL模型
model = pymodel.java   # java函数，可以将命令语句直接转换为Java模型对象，
                       # 以便可以使用Java模型对象所提供的功能(就是可以在comsol中做一个操作，然后把模型另存为java模式，打开文本文件后最下面一行的代码就是刚刚操作所对应的java代码，可以直接复制粘贴过来用)

In [177]:
# 生成磁芯编号矩阵，用于材料的选择  _b代表下方磁芯，_t代表上方磁芯
import numpy as np

# 最后一行磁芯的编号
bottom_row_b = np.array([2, 22, 40, 58, 76, 96, 114, 132, 150])    # 加铝板的是 bottom_row_b_Al = np.array[4, 24, 42, 60, 78, 98, 116, 134, 152]
bottom_row_t = np.array([3, 23, 41, 59, 77, 97, 115, 133, 151]) 

# 生成一个初始值为0的9x9矩阵
core_b_num = np.zeros((9, 9))
core_t_num = np.zeros((9, 9))

# 从最后一行开始，每行递增2
for i in range(8, -1, -1):
    core_b_num[i, :] = bottom_row_b + (8 - i) * 2
    core_t_num[i, :] = bottom_row_t + (8 - i) * 2

core_b_num = core_b_num.astype(int)
core_t_num = core_t_num.astype(int)
print(core_t_num)


[[ 19  39  57  75  93 113 131 149 167]
 [ 17  37  55  73  91 111 129 147 165]
 [ 15  35  53  71  89 109 127 145 163]
 [ 13  33  51  69  87 107 125 143 161]
 [ 11  31  49  67  85 105 123 141 159]
 [  9  29  47  65  83 103 121 139 157]
 [  7  27  45  63  81 101 119 137 155]
 [  5  25  43  61  79  99 117 135 153]
 [  3  23  41  59  77  97 115 133 151]]


In [178]:
# 假设 1：Mat 1; 2：Mat 2; 3：Mat 3; 4：Mat 4
# 假设优化右下角的16块儿磁芯，假设一个右下角的4*4磁芯材料参数矩阵
bottom_righthand_materials = np.array([[4, 4, 2, 2,], [3, 3, 2, 2], [2, 2, 2, 2], [2, 2, 2, 2]])

# 将右下角的磁芯材料参数矩阵扩展到整个磁芯平面

core_materials = np.zeros((9, 9)) # 生成一个初始的9x9矩阵

# 9*9的磁芯平面上，第五行和第五列两条中心线的磁芯材料参数是默认不变的
given_row = np.array([2, 2, 4, 4, 1, 4, 4, 2, 2])
given_column = np.array([2, 2, 3, 3, 1, 3, 3, 2, 2])

core_materials[4, :] = given_row          # 将给定的值赋予第五行和第五列
core_materials[:, 4] = given_column

# 将右下角的磁芯矩阵进行旋转扩展
bottom_lefthand_materials = np.copy(np.rot90(bottom_righthand_materials, k=-1))
top_lefthand_materials = np.copy(np.rot90(bottom_righthand_materials, k=-2))
top_righthand_materials = np.copy(np.rot90(bottom_righthand_materials, k=-3))

for i in range(4):
    for j in range(4):
        if bottom_lefthand_materials[i, j] == 3:
            bottom_lefthand_materials[i, j] = 4            
        elif bottom_lefthand_materials[i, j] == 4:
            bottom_lefthand_materials[i, j] = 3
            
        if top_righthand_materials[i, j] == 3:
            top_righthand_materials[i, j] = 4
        elif top_righthand_materials[i, j] == 4:
            top_righthand_materials[i, j] = 3

core_materials[0:4, 0:4] = top_lefthand_materials
core_materials[0:4, 5:9] = top_righthand_materials
core_materials[5:9, 0:4] = bottom_lefthand_materials
core_materials[5:9, 5:9] = bottom_righthand_materials
core_materials = core_materials.astype(int)

# 将材料信息对应到磁芯编号，以方便对comsol中材料进行修改
mat1 = []
mat2 = []
mat3 = []
mat4 = []
for i in range(9):
    for j in range(9):
        eval('mat'+str(core_materials[i, j])).append(core_b_num[i, j])
        eval('mat'+str(core_materials[i, j])).append(core_t_num[i, j])
        blk_b = "blk"+str(1+j+(9-(i+1))*9)
        blk_t = "blk"+str(81+1+j+(9-(i+1))*9) 
        pos_3 = [str((j-4)*50 - 25), str((i-4)*50 - 25), "distance/2+5.6-2.5+2"]
        pos_5 = [str((j-4)*50 - 25), str((i-4)*50 - 25), "distance/2+5.6-2.5"]
        if core_materials[i, j] in [3,4]:
            model.component("comp1").geom("geom1").feature(blk_b).set("size", ['50', '50', '3'])     # 修改几何结构，不同材料厚度不同
            model.component("comp1").geom("geom1").feature(blk_t).set("size", ['50', '50', '3'])
            model.component("comp1").geom("geom1").feature(blk_t).set("pos", pos_3) # 上方小磁芯的基准坐标也要修改
        if core_materials[i, j] in [1,2]:
            model.component("comp1").geom("geom1").feature(blk_b).set("size", ['50', '50', '5'])
            model.component("comp1").geom("geom1").feature(blk_t).set("size", ['50', '50', '5'])
            model.component("comp1").geom("geom1").feature(blk_t).set("pos", pos_5)

model.save(path)    

In [156]:
mph.tree(pymodel)      # 查看模型树，可用于确定要修改的节点的位置

优化模型 - 副本
├─ parameters
│  └─ 参数 1
├─ functions
├─ components
│  └─ 组件 1
├─ geometries
│  └─ 几何 1
│     ├─ core1
│     ├─ core2
│     ├─ core3
│     ├─ core4
│     ├─ core5
│     ├─ core6
│     ├─ core7
│     ├─ core8
│     ├─ core9
│     ├─ core10
│     ├─ core11
│     ├─ core12
│     ├─ core13
│     ├─ core14
│     ├─ core15
│     ├─ core16
│     ├─ core17
│     ├─ core18
│     ├─ core19
│     ├─ core20
│     ├─ core21
│     ├─ core22
│     ├─ core23
│     ├─ core24
│     ├─ core25
│     ├─ core26
│     ├─ core27
│     ├─ core28
│     ├─ core29
│     ├─ core30
│     ├─ core31
│     ├─ core32
│     ├─ core33
│     ├─ core34
│     ├─ core35
│     ├─ core36
│     ├─ core37
│     ├─ core38
│     ├─ core39
│     ├─ core40
│     ├─ core41
│     ├─ core42
│     ├─ core43
│     ├─ core44
│     ├─ core45
│     ├─ core46
│     ├─ core47
│     ├─ core48
│     ├─ core49
│     ├─ core50
│     ├─ core51
│     ├─ core52
│     ├─ core53
│     ├─ core54
│     ├─ core55
│     ├─ core56
│     ├─ core57

In [179]:
(pymodel/'materials'/'Mat 3').selection()     # 查询现在Mat 3所包含的几何编号或节点

array([80, 81, 82, 83, 86, 87, 88, 89])

In [180]:
(pymodel/'materials'/'Mat 1').select(mat1)
(pymodel/'materials'/'Mat 2').select(mat2)
(pymodel/'materials'/'Mat 3').select(mat3)
(pymodel/'materials'/'Mat 4').select(mat4)
model.save(path)                                 # 保存修改

In [115]:
(pymodel/'selection'/'core6').properties()      # 查看某个节点可以修改或查看的属性

{}

In [171]:
pymodel.studies()       # 查看能运行的研究

['研究 1']

In [173]:
pymodel.solve('研究 1')    # 运行研究
# pymodel.evaluate('mf.LCoil_1','H')   # 返回计算结果-自感
pymodel.evaluate('abs(mf.PhiCoil_2/mf.ICoil_1)/sqrt(mf.LCoil_1*mf.LCoil_1)','1')  # 返回计算结果-耦合系数

array(0.26837731)

In [181]:
client.remove(pymodel)    # 释放模型                             
client.clear()